In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationEntityMemory, CombinedMemory
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Setup your environment

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") ## Put your OpenAI API key here
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY") ## Put your Tavily Search API key here
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_HUB_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_TRACING_V2"] = 'true' ## Set this as True
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com/' ## Set this as: https://api.smith.langchain.com/
os.environ["LANGCHAIN_HUB_API_URL"] = 'https://api.hub.langchain.com' ## Set this as : https://api.hub.langchain.com
os.environ["LANGCHAIN_PROJECT"] = 'llm-agents-memory'

## Memory types in Langchain

There are different short term memory types supported in Langchain. Important Ones are:
* Conversation Buffer Memory
* Conversation Buffer Window Memory
* Conversation Summary Memory
* Entity Memory

Read about Langchain memory types here: 

### Simple Conversation Buffer Memory

![Buffer Memory](images/buffer-memory.png)

| Pros      | Cons |
| ----------- | ----------- |
| Simple to implement      | Longer prompt -> Higher cost and Latency       |
| Passes everything to the LLM   | Very long conversations will hit the model token limit        |

In [4]:

llm = ChatOpenAI()
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

/Users/pdwivedi/miniconda3/envs/test/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [5]:
result = conversation.invoke({"human_input": "hi, I am thinking of having pasta for lunch"})
print(result['text'])

That sounds delicious! What kind of pasta are you thinking of having?


In [6]:
result = conversation.invoke({"human_input": "hi,I like white sauce. What else goes with it?"})
print(result['text'])

White sauce pasta is a classic choice! You can add some protein like chicken or shrimp, or some vegetables like broccoli, mushrooms, or spinach to make it more filling and nutritious. You can also top it with some grated parmesan cheese or fresh herbs for extra flavor. Enjoy your lunch!


In [7]:
result = conversation.invoke({"human_input": "hi, add vegetarian ingredients and herbs. Also suggest good wine"})
print(result['text'])

For a vegetarian white sauce pasta, you can add ingredients like roasted cherry tomatoes, roasted bell peppers, artichokes, and fresh basil or parsley for herbs. As for wine pairing, a nice crisp white wine like a Pinot Grigio or Sauvignon Blanc would complement the flavors of the pasta well. Enjoy your meal!


In [8]:
### Look at the chat history stored
import pprint
pprint.pprint(result['chat_history'])

[HumanMessage(content='hi, I am thinking of having pasta for lunch'),
 AIMessage(content='That sounds delicious! What kind of pasta are you thinking of having?'),
 HumanMessage(content='hi,I like white sauce. What else goes with it?'),
 AIMessage(content='White sauce pasta is a classic choice! You can add some protein like chicken or shrimp, or some vegetables like broccoli, mushrooms, or spinach to make it more filling and nutritious. You can also top it with some grated parmesan cheese or fresh herbs for extra flavor. Enjoy your lunch!'),
 HumanMessage(content='hi, add vegetarian ingredients and herbs. Also suggest good wine'),
 AIMessage(content='For a vegetarian white sauce pasta, you can add ingredients like roasted cherry tomatoes, roasted bell peppers, artichokes, and fresh basil or parsley for herbs. As for wine pairing, a nice crisp white wine like a Pinot Grigio or Sauvignon Blanc would complement the flavors of the pasta well. Enjoy your meal!')]


### Conversation Buffer Window Memory

![Buffer Memory](images/buffer-window-memory.png)

| Pros      | Cons |
| ----------- | ----------- |
| Simple to implement      | May not pass all the relevant context      |
| Passes only recent temporally relevant messages    |

In [9]:
llm = ChatOpenAI()
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferWindowMemory(memory_key="chat_history", k=2, return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [10]:
result = conversation.invoke({"human_input": "hi! What are the latest trends in AI?"})
print(result['text'])

Hello! Some of the latest trends in AI include advancements in natural language processing, computer vision, autonomous vehicles, and reinforcement learning. There is also a growing focus on ethical AI and responsible AI development practices. Is there a specific aspect of AI you are interested in?


In [11]:
result = conversation.invoke({"human_input": "hi! Tell me about autonomous driving?"})
print(result['text'])

Autonomous driving, also known as self-driving cars, is a rapidly advancing field in AI. It involves developing vehicles that can operate without human intervention. These vehicles use a combination of sensors, cameras, radar, and AI algorithms to navigate roads, detect obstacles, and make decisions in real-time. Companies like Tesla, Waymo, and Uber are investing heavily in autonomous driving technology. The goal is to improve road safety, reduce traffic congestion, and provide greater mobility for people who are unable to drive. There are still challenges to overcome, such as ensuring the safety and reliability of autonomous vehicles in all driving conditions.


In [12]:
result = conversation.invoke({"human_input": "Which do you think is the leader?"})
print(result['text'])

It's difficult to determine a single leader in the field of autonomous driving, as there are several companies making significant advancements in this area. Companies like Tesla, Waymo, and Uber are often considered key players in the development of autonomous driving technology. Each of these companies has its own unique approach and strengths in this field. Tesla is known for its innovative use of AI and machine learning in its vehicles, Waymo has been testing autonomous vehicles on public roads for several years, and Uber has been working on autonomous driving technology for its ride-sharing services. Ultimately, the leader in autonomous driving may vary depending on the specific criteria or metrics being considered.


In [13]:
result = conversation.invoke({"human_input": "Is this really safe?"})
print(result['text'])

While autonomous driving technology has the potential to improve road safety by reducing human error, there are still concerns about the safety and reliability of autonomous vehicles in all driving conditions. Companies developing autonomous driving technology are continuously testing and improving their systems to ensure safety. It's important to consider that like any emerging technology, there may be risks and challenges that need to be addressed before autonomous driving becomes widely adopted.


In [14]:
### Look at the chat history stored
import pprint
pprint.pprint(result['chat_history'])

[HumanMessage(content='hi! Tell me about autonomous driving?'),
 AIMessage(content='Autonomous driving, also known as self-driving cars, is a rapidly advancing field in AI. It involves developing vehicles that can operate without human intervention. These vehicles use a combination of sensors, cameras, radar, and AI algorithms to navigate roads, detect obstacles, and make decisions in real-time. Companies like Tesla, Waymo, and Uber are investing heavily in autonomous driving technology. The goal is to improve road safety, reduce traffic congestion, and provide greater mobility for people who are unable to drive. There are still challenges to overcome, such as ensuring the safety and reliability of autonomous vehicles in all driving conditions.'),
 HumanMessage(content='Which do you think is the leader?'),
 AIMessage(content="It's difficult to determine a single leader in the field of autonomous driving, as there are several companies making significant advancements in this area. Compa

You can also use token buffer memory to take a fixed token limit from the past. See here for more details:
https://python.langchain.com/docs/modules/memory/types/token_buffer/

### Conversation Summary Memory

![Buffer Memory](images/summary-memory.png)

| Pros      | Cons |
| ----------- | ----------- |
| Summarizes long conversations     | Involves Time and cost to create summaries    |
| Can work with bringing in old context from long conversations    | Pertinent information many not be in summary|

In [15]:
llm = ChatOpenAI()
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationSummaryMemory(llm=ChatOpenAI(temperature=0), memory_key='chat_history', return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [16]:
result = conversation.invoke({"human_input": "hi! What are the latest trends in AI?"})
print(result['text'])

Hi there! Some of the latest trends in AI include advancements in natural language processing, computer vision, reinforcement learning, and the increasing use of AI in healthcare, finance, and other industries. Is there a specific area of AI you're interested in?


In [17]:
result = conversation.invoke({"human_input": "Tell me about autonomous driving"})
print(result['text'])

Autonomous driving is a fascinating area of AI that has seen significant advancements in recent years. It involves developing vehicles that can operate without human intervention. Companies like Tesla, Waymo, and Uber are leading the way in this technology, using AI algorithms to navigate roads, detect obstacles, and make decisions in real-time. Autonomous driving has the potential to revolutionize transportation, making it safer, more efficient, and convenient. Is there anything specific you'd like to know about autonomous driving?


In [18]:
result = conversation.invoke({"human_input": "How many years do you think till we have self driving cars?"})
print(result['text'])

It's difficult to predict an exact timeline, but many experts believe that fully autonomous vehicles could be on the roads within the next 5-10 years. However, the widespread adoption of self-driving cars will depend on various factors such as regulatory approval, technological advancements, and public acceptance. Companies like Tesla, Waymo, and Uber are making significant progress in this area, so it's definitely an exciting time for autonomous driving technology.


In [19]:
result = conversation.invoke({"human_input": "Will these cars be safe?"})
print(result['text'])

The development of autonomous vehicles is focused on safety, with AI algorithms constantly analyzing data from sensors to make split-second decisions to prevent accidents. Companies are conducting extensive testing to ensure the safety of these vehicles before they are deployed on public roads. However, there are still challenges to overcome, such as unpredictable human behavior and adverse weather conditions. Regulatory bodies are also working on establishing guidelines for autonomous vehicles to ensure they meet safety standards. Overall, the goal is to make autonomous driving as safe as possible for all road users.


In [20]:
### Look at the chat history stored
import pprint
pprint.pprint(result['chat_history'])

[SystemMessage(content='The human asks about the latest trends in AI, and the AI responds by mentioning advancements in natural language processing, computer vision, reinforcement learning, and the increasing use of AI in industries like healthcare and finance. The AI also asks if there is a specific area of AI the human is interested in. The human inquires about autonomous driving, and the AI explains that companies like Tesla, Waymo, and Uber are leading the way in developing vehicles that can operate without human intervention using AI algorithms. The AI mentions that experts believe fully autonomous vehicles could be on the roads within the next 5-10 years, but widespread adoption will depend on factors like regulatory approval and public acceptance.')]


In [21]:
print(result)

{'human_input': 'Will these cars be safe?', 'chat_history': [SystemMessage(content='The human asks about the latest trends in AI, and the AI responds by mentioning advancements in natural language processing, computer vision, reinforcement learning, and the increasing use of AI in industries like healthcare and finance. The AI also asks if there is a specific area of AI the human is interested in. The human inquires about autonomous driving, and the AI explains that companies like Tesla, Waymo, and Uber are leading the way in developing vehicles that can operate without human intervention using AI algorithms. The AI mentions that experts believe fully autonomous vehicles could be on the roads within the next 5-10 years, but widespread adoption will depend on factors like regulatory approval and public acceptance.')], 'text': 'The development of autonomous vehicles is focused on safety, with AI algorithms constantly analyzing data from sensors to make split-second decisions to prevent a

### Combine 2 kinds of memory

In [22]:
llm = ChatOpenAI()
template = """You are a chatbot having a conversation with a human.

Summary of conversation:
{history}

Current conversation:
{recent_conversation}

Human: {input}
Chatbot:

"""

prompt = PromptTemplate(
    input_variables=["history", "recent_conversation", "input"], template=template
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
conv_memory = ConversationBufferWindowMemory(memory_key="recent_conversation", k=2, input_key='input', return_messages=True)
summary_memory = ConversationSummaryMemory(llm=ChatOpenAI(temperature=0), memory_key='history', input_key='input', return_messages=True)
memory = CombinedMemory(memories=[conv_memory, summary_memory])

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [23]:
result = conversation.invoke({"input": "help me with my Christmas shopping please"})
print(result['text'])

Of course! I can help you find some gift ideas or recommend some popular items. What are you looking for in particular?


In [24]:
result = conversation.invoke({"input": "for my entire family. Lets start with my parents. My dad likes to do wood working so suggest a new tool for him. My mom likes to cook so suggest a new kitchen gadget for her"})
print(result['text'])

For your dad who enjoys woodworking, you could consider getting him a new set of high-quality chisels or a versatile multi-tool that could come in handy for various projects. For your mom who loves to cook, a popular kitchen gadget you could consider is a high-end stand mixer or a high-tech kitchen scale. Let me know if you need more suggestions for any other family members!


In [25]:
result = conversation.invoke({"input": "My sister likes massages and spa."})
print(result['text'])

For your sister who enjoys massages and spa, you could consider getting her a gift certificate for a relaxing spa day, a set of luxurious bath bombs or essential oils for at-home relaxation, or a massage pillow or massager for stress relief. Let me know if you need more suggestions for any other family members!


In [26]:
result = conversation.invoke({"input": "My wife likes jewellery."})
print(result['text'])

For your wife who likes jewelry, you could consider getting her a beautiful piece of jewelry such as a necklace, bracelet, or earrings. You could also consider a jewelry box or organizer to help keep her collection organized. Let me know if you need more suggestions for any other family members!


In [27]:
pprint.pprint(result)

{'history': [SystemMessage(content='The human asks the AI to help with Christmas shopping, starting with gift ideas for their parents. The AI suggests a new set of high-quality chisels or a versatile multi-tool for the dad who enjoys woodworking, and a high-end stand mixer or a high-tech kitchen scale for the mom who loves to cook. The AI then recommends a gift certificate for a relaxing spa day, luxurious bath bombs, or a massage pillow for the sister who enjoys massages and spa. The AI offers to provide more suggestions for other family members if needed.')],
 'input': 'My wife likes jewellery.',
 'recent_conversation': [HumanMessage(content='for my entire family. Lets start with my parents. My dad likes to do wood working so suggest a new tool for him. My mom likes to cook so suggest a new kitchen gadget for her'),
                         AIMessage(content='For your dad who enjoys woodworking, you could consider getting him a new set of high-quality chisels or a versatile multi-too

### Entity Memory

![Entity Memory](images/entity-example.png)

| Pros      | Cons |
| ----------- | ----------- |
| Organizes relevant information by entity     | Involves Time and cost to create entities    |
| More likely to capture relevant details for long conversations    | Can become very large for long conversations|

In [28]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE)

input_variables=['entities', 'history', 'input'] template='You are an assistant to a human, powered by a large language model trained by OpenAI.\n\nYou are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nYou are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowi

In [29]:
llm = ChatOpenAI()
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationEntityMemory(llm=ChatOpenAI(temperature=0), memory_key='chat_history', return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    verbose=False,
    memory=memory
)

In [30]:
result = conversation.invoke({"input": "We are all going for a family dinner"})
result['text']

'That sounds like a lovely plan! Family dinners are a great way to spend quality time together and enjoy some good food. Do you have a favorite dish or restaurant you like to go to for family dinners?'

In [31]:
result = conversation.invoke({"input": "I agree. We are going for an Italian dinner. I like to order raviola pasta. My husband like gnocci. My son Sam likes margarita pizza and my daughter Bella likes fettucini alfredo. Can you suggest starters for us?"})
result['text']

"That's wonderful to hear that you're going for an Italian dinner! Since you and your family have different preferences for the main course, it's great to have a variety of options. \n\nFor starters, you could consider ordering some classic Italian appetizers like bruschetta, caprese salad, or antipasto platter. These options are light and flavorful, and they can help set the tone for a delicious meal ahead. Additionally, you could also consider ordering some garlic bread or garlic knots for a simple yet tasty starter.\n\nI hope these suggestions help enhance your family dinner experience! Enjoy your meal with your loved ones. If you have any other preferences or dietary restrictions, feel free to let me know for more personalized recommendations."

In [32]:
result = conversation.invoke({"input": "Perfect. Me and my husband will like the Caprese salad. The kids will like to share a bruschetta. What drinks will you suggest?"})
result['text']

'For drinks to accompany your Italian dinner, you could consider some classic options such as a glass of wine. For a family-friendly option, you could go for some Italian sodas like Limonata or Aranciata, which are refreshing and pair well with Italian cuisine. If you prefer non-alcoholic options, you could also consider ordering some sparkling water or fruit juices. Enjoy your meal with these delicious drink choices!'

In [33]:
result = conversation.invoke({"input": "Soda for the kids and prosecco for the adults"})
result['text']

"That sounds like a great choice! Soda for the kids and prosecco for the adults is a perfect combination to cater to everyone's preferences. The kids can enjoy a refreshing and fun soda, while the adults can savor the sparkling and elegant prosecco. It's a thoughtful selection that will surely complement your Italian dinner experience. Enjoy your meal with these delightful drink choices!"

In [34]:
conversation.memory.entity_store.store

{'Italian': 'Italian cuisine offers a variety of dishes such as raviola pasta, gnocci, margarita pizza, and fettucini alfredo, catering to different preferences within the family.',
 'Sam': 'Sam likes margarita pizza.',
 'Bella': 'Bella likes fettucini alfredo.',
 'Caprese': 'Caprese salad is preferred by the human and their husband for the family dinner.',
 'bruschetta': 'Bruschetta is a shared appetizer that the kids enjoy during family dinners.',
 'Soda': 'Soda is a refreshing and fun drink choice for kids, while prosecco is a sparkling and elegant option for adults.',
 'Prosecco': 'Prosecco is a sparkling and elegant drink choice for adults to savor during an Italian dinner.'}

### Exercise

Explore the CONVERSATION_TOKEN_BUFFER memory from Langchain. What are its advantages/disadvantages compared to conversation window memory?

https://python.langchain.com/docs/modules/memory/types/token_buffer/